In [4]:
# -*- coding: utf-8 -*-  
import os, glob
import Tkinter as tk
from PIL import Image, ImageTk
#from utils import get_jpgs_d0
import xml.dom.minidom as xmldom

######################
# button for change: https://stackoverflow.com/questions/41657449/tkinter-not-changing-image-on-button-press
# global variable: https://morvanzhou.github.io/tutorials/python-basic/tkinter/2-01-label-button/
# pack layout: https://blog.csdn.net/liuxu0703/article/details/54428405
# not Imagetk: sudo apt install python-imaging-tk
# pyimage: restart, tk.Toplevel()
# xml: https://www.cnblogs.com/xiaobingqianrui/p/8405813.html
# xml: https://www.cnblogs.com/wcwnina/p/7222180.html
######################
def next_img_r():
    global r_dir
    global r_idx, l_idx
    global ImgH, ImgW
    global Ratio
    r_idx += 1
    if r_idx == len(r_pths):
        print('recycling')
        r_idx = 0
    Ratio = SrcR
    root.r_img = ImageTk.PhotoImage(Image.open(os.path.join(r_dir, r_pths[r_idx])).resize((ImgW, ImgH)))
    r_label.configure(image=root.r_img)
    b_large.configure(text=str(int(Ratio*100))+'%')
    print('updated')

        
def enlarge_img():
    global Ratio, Rstep
    global PosL, PosU, PosR, PosD
    Ratio += Rstep
    PosL = (int(SrcW * Ratio) - ImgW) / 2
    PosU = (int(SrcH * Ratio) - ImgH) / 2
    PosR = PosL + ImgW - 1
    PosD = PosU + ImgH - 1
    root.r_img = ImageTk.PhotoImage(Image.open(os.path.join(r_dir, r_pths[r_idx])).
                                    resize((int(SrcW * Ratio), int(SrcH * Ratio))).crop((PosL, PosU, PosR, PosD)))
    r_label.configure(image=root.r_img)
    b_large.configure(text=str(int(Ratio*100))+'%')
    print('enlarge')

    
def print_dict(grps):
    for key in grps.keys():
        str_ = '[' + unicode(key, encoding="utf-8") + ']={'
        for itm in grps[key]:
            str_ += unicode(itm, encoding="utf-8") + ','
        str_ += '}'
        print(str_)

        
def get_cls_dict(prts):
    cls_dict = {}
    img_dict = {}
    for tmp_prt in prts:
        tmp_dirs = glob.glob(tmp_prt + '/*')
        for tmp_dir in tmp_dirs:
            tmp_cls = tmp_dir[len(tmp_prt) + 1:]
            if tmp_cls not in cls_dict:
                cls_dict[tmp_cls] = [tmp_dir]
            else:
                cls_dict[tmp_cls].append(tmp_dir)
            img_dict[tmp_dir] = glob.glob(tmp_dir + '/*.jpg')
    return cls_dict, img_dict


def get_bbox_xml(xml_pth):
    obj_dict = {}
    domobj = xmldom.parse(xml_pth)    
    node_objs = domobj.documentElement.getElementsByTagName("object")    
    for i in range(len(node_objs)):
        #print node_objs[i].nodeName
        node_name = node_objs[i].getElementsByTagName("name")        
        node_bbox = node_objs[i].getElementsByTagName("bndbox")
        node_xmin = node_bbox[0].getElementsByTagName("xmin")
        node_ymin = node_bbox[0].getElementsByTagName("ymin")
        node_xmax = node_bbox[0].getElementsByTagName("xmax")
        node_ymax = node_bbox[0].getElementsByTagName("ymax")
        name = node_name[0].firstChild.data.encode("utf-8")     #unicode->utf8
        xmin = int(node_xmin[0].firstChild.data)
        ymin = int(node_ymin[0].firstChild.data)
        xmax = int(node_xmax[0].firstChild.data)
        ymax = int(node_ymax[0].firstChild.data)
        if name not in obj_dict:
            obj_dict[name] = [[xmin, ymin, xmax, ymax]]
        else:
            obj_dict[name].append([xmin, ymin, xmax, ymax])
    return obj_dict


def get_lr_indices():
    #global def_grps, cls_dict, img_dict
    global l_key_idx, l_dir_idx, l_img_idx
    global r_key_idx, r_dir_idx, r_img_idx
    global l_key_cur, l_dir_cur, l_img_cur
    global r_key_cur, r_dir_cur, r_img_cur
    l_key_cur = def_grps.keys()[l_key_idx]
    l_dir_cur = cls_dict[l_key_cur][l_dir_idx]
    l_img_cur = img_dict[l_dir_cur][l_img_idx]
    r_key_cur = def_grps[l_key_cur][r_key_idx]
    r_dir_cur = cls_dict[r_key_cur][r_dir_idx]
    r_img_cur = img_dict[r_dir_cur][r_img_idx]

    
def next_key_l():
    global l_key_idx
    l_key_idx += 1
    if l_key_idx == len(def_grps.keys()):
        l_key_idx = 0
    l_dir_idx, l_img_idx = 0, 0
    r_key_idx, r_dir_idx, r_img_idx = 0, 0, 0
    get_lr_indices()
    
    global Ratio    
    Ratio = SrcR    
    
    #b_large.configure(text=str(int(Ratio*100))+'%')
    load_img_l('<next_key_l> updated for: ')
    load_img_r('<next_key_l> updated for: ')

    
def next_key_r():
    global r_key_idx
    r_key_idx += 1
    if r_key_idx == len(def_grps[l_key_cur]):
        r_key_idx = 0
    
    r_dir_idx, r_img_idx = 0, 0
    get_lr_indices()
    load_img_r('<next_key_r> updated for: ')

    
def next_dir_l():
    global l_dir_idx
    l_dir_idx += 1
    if l_dir_idx == len(cls_dict[l_key_cur]):
        l_dir_idx = 0
    l_img_idx = 0
    get_lr_indices()
    load_img_l('<next_dir_l> updated for: ')

    
def next_dir_r():
    global r_dir_idx
    r_dir_idx += 1
    if r_dir_idx == len(cls_dict[r_key_cur]):
        r_dir_idx = 0
    r_img_idx = 0
    get_lr_indices()
    load_img_r('<next_dir_r> updated for: ')

    
def next_img_l():
    global l_img_idx
    l_img_idx += 1
    if l_img_idx == len(img_dict[l_dir_cur]):
        l_img_idx = 0
    get_lr_indices()    
    load_img_l('<next_img_l> updated for: ')


def next_img_r():
    global r_img_idx
    r_img_idx += 1
    if r_img_idx == len(img_dict[r_dir_cur]):
        r_img_idx = 0
    get_lr_indices()    
    load_img_r('<next_img_r> updated for: ')


def get_img(img_pth, obj_key=''):
    img0 = Image.open(img_pth)   
    xml_pth = img_pth[0:-4] + '.xml'
    obj_dict = get_bbox_xml(xml_pth)
    if (obj_key != '') and (obj_key not in obj_dict):
        return img0.resize((ImgW, ImgH))
    else:
        if obj_key != '':
            obj_pos = obj_dict[obj_key][0]
        else:
            obj_pos = obj_dict[obj_dict.keys()[0]][0]
        x1 = (obj_pos[0] + obj_pos[2]) / 2 - ImgW
        if x1 < 0:
            x1 = 0
        y1 = (obj_pos[1] + obj_pos[3]) / 2 - ImgH
        if y1 < 0:
            y1 = 0
        return img0.crop((x1, y1, ImgW, ImgH))
    
    
def load_img_l(aff_str, obj_key=''):
    root.l_img = ImageTk.PhotoImage(get_img(l_img_cur, obj_key))
    l_label.configure(image=root.l_img)
    print aff_str + l_img_cur.decode('string_escape')

    
def load_img_r(aff_str, obj_key=''):
    root.r_img = ImageTk.PhotoImage(get_img(r_img_cur, obj_key))
    r_label.configure(image=root.r_img)
    print aff_str + r_img_cur.decode('string_escape')


#obj_dict = get_bbox_xml('/media/gait/DATA/04data/01competitions/11tianchi_bupi/xuelang_round1_train_part3_20180709/吊经/J01_2018.06.28 15_19_28.xml')
#print(obj_dict)

root = tk.Tk()
#root = tk.Toplevel()

# s0: defined groups
def_grps = {}
def_grps['吊弓'] = ['吊弓', '吊纬', '吊经']
def_grps['剪洞'] = ['剪洞', '扎洞', '边扎洞', '擦洞', '毛洞', '破洞', '结洞']
print(def_grps)
print_dict(def_grps)

# s1: get image dirs and class dirs
cls_dict, img_dict = get_cls_dict(['xuelang_round1_train_part1_20180628', 
                          'xuelang_round1_train_part2_20180705',
                          'xuelang_round1_train_part3_20180709'])
#print_dict(cls_dict)
#print_dict(img_dict)

# s2: set the left and right indices of key words
l_key_idx, l_dir_idx, l_img_idx = 0, 0, 0
r_key_idx, r_dir_idx, r_img_idx = 0, 0, 0
l_key_cur, l_dir_cur, l_img_cur = '', '', ''
r_key_cur, r_dir_cur, r_img_cur = '', '', ''


# s3: set the image size for reading and showing 
SrcW, SrcH = 2560, 1920
SrcR = 0.3
ImgW, ImgH = int(SrcW * SrcR), int(SrcH * SrcR)
PosL, PosU, PosR, PosD = 0, 0, ImgW-1, ImgH-1
Ratio = SrcR
Rstep = 0.2

# s4: add the left and right label
get_lr_indices()
#print(l_img_cur)
#print(r_img_cur)
root.l_img = ImageTk.PhotoImage(Image.open(l_img_cur).resize((ImgW, ImgH)))
l_label = tk.Label(root, image=root.l_img)
l_label.pack(side=tk.LEFT)
root.r_img = ImageTk.PhotoImage(Image.open(r_img_cur).resize((ImgW, ImgH)))
r_label = tk.Label(root, image=root.r_img)
r_label.pack(side=tk.LEFT)

# s5: add buttons
b_next_key_l = tk.Button(root, text="NextKeyL", command=next_key_l)
b_next_key_l.pack()
b_next_dir_l = tk.Button(root, text="NextDirL", command=next_dir_l)
b_next_dir_l.pack()
b_next_img_l = tk.Button(root, text="NextImgL", command=next_img_l)
b_next_img_l.pack()
b_next_key_r = tk.Button(root, text="NextKeyR", command=next_key_r)
b_next_key_r.pack()
b_next_dir_r = tk.Button(root, text="NextDirR", command=next_dir_r)
b_next_dir_r.pack()
b_next_img_r = tk.Button(root, text="NextImgR", command=next_img_r)
b_next_img_r.pack()

'''
# s2: add buttons


b_large = tk.Button(root, text=str(int(Ratio*100))+'%', command=enlarge_img)
b_large.pack()
'''

root.mainloop()



{'\xe5\x90\x8a\xe5\xbc\x93': ['\xe5\x90\x8a\xe5\xbc\x93', '\xe5\x90\x8a\xe7\xba\xac', '\xe5\x90\x8a\xe7\xbb\x8f'], '\xe5\x89\xaa\xe6\xb4\x9e': ['\xe5\x89\xaa\xe6\xb4\x9e', '\xe6\x89\x8e\xe6\xb4\x9e', '\xe8\xbe\xb9\xe6\x89\x8e\xe6\xb4\x9e', '\xe6\x93\xa6\xe6\xb4\x9e', '\xe6\xaf\x9b\xe6\xb4\x9e', '\xe7\xa0\xb4\xe6\xb4\x9e', '\xe7\xbb\x93\xe6\xb4\x9e']}
[吊弓]={吊弓,吊纬,吊经,}
[剪洞]={剪洞,扎洞,边扎洞,擦洞,毛洞,破洞,结洞,}


<next_key_l> updated for: xuelang_round1_train_part2_20180705/剪洞/J01_2018.06.22 08_45_25.jpg
<next_key_l> updated for: xuelang_round1_train_part2_20180705/剪洞/J01_2018.06.22 08_45_25.jpg


<next_key_l> updated for: xuelang_round1_train_part2_20180705/吊弓/J01_2018.06.16 10_51_31.jpg
<next_key_l> updated for: xuelang_round1_train_part2_20180705/吊弓/J01_2018.06.16 10_51_31.jpg


<next_dir_l> updated for: xuelang_round1_train_part3_20180709/吊弓/J01_2018.06.17 14_37_37.jpg


<next_key_l> updated for: xuelang_round1_train_part3_20180709/剪洞/J01_2018.06.23 09_09_55.jpg
<next_key_l> updated for: xuelang_round1_train_part2_20180705/剪洞/J01_2018.06.22 08_45_25.jpg


<next_key_l> updated for: xuelang_round1_train_part3_20180709/吊弓/J01_2018.06.17 14_37_37.jpg
<next_key_l> updated for: xuelang_round1_train_part2_20180705/吊弓/J01_2018.06.16 10_51_31.jpg
